In [2]:
import pandas as pd
import numpy as np

In [7]:
#carga del archivo
df_centroids = pd.read_excel('./results/events_upz_month1.xlsx', encoding='utf-8', error_bad_lines=False)
df_centroids

,Unnamed: 0,COD_UPZ,UPZ,MES,ANIO,NORM,CXLAT,CXLON
0,0,UPR1,UPR ZONA NORTE,1,2014,0.173554,4.760959,-74.083744
1,1,UPR1,UPR ZONA NORTE,2,2014,0.338843,4.761651,-74.083329
2,2,UPR1,UPR ZONA NORTE,3,2014,0.636364,4.761338,-74.083295
3,3,UPR1,UPR ZONA NORTE,4,2014,0.297521,4.761111,-74.083503
4,4,UPR1,UPR ZONA NORTE,5,2014,0.595041,4.761188,-74.083482
...,...,...,...,...,...,...,...,...
8022,8022,UPZ99,CHAPINERO,7,2019,0.387755,4.640466,-74.064755
8023,8023,UPZ99,CHAPINERO,8,2019,0.557823,4.640799,-74.064713
8024,8024,UPZ99,CHAPINERO,9,2019,0.272109,4.640554,-74.064910
8025,8025,UPZ99,CHAPINERO,10,2019,0.530612,4.640667,-74.064735


In [8]:
from geojson import Feature, Point, FeatureCollection

# centroids SKDE by month
# función para colocar la cantidad de incidentes de nuse durante un año (depende del filtro)
# parámetro dataframe df_centroids
def generate_centroids_by_month(dfin):
    
    # lista de elementos que van el archivo geojson
    lst_features = list()
    for i in range (1, 13):
        

        dfout = dfin[dfin['MES']== i]
        for j in range (len(dfout)):
            
            '''creando una geometría de tipo punto geojson (este es el centroide del 
            polígono de la upz en longitud y latitud)'''
            i_lnglat = Point((dfout['CXLON'].iloc[j], dfout['CXLAT'].iloc[j]))
            
            # mes para el incidente
            mon = int(dfout['MES'].iloc[j])
            # cantidad de incidentes
            inc = float(dfout['NORM'].iloc[j])
            '''
            creando un objeto de tipo geojson al que se le añade la geometría y las propiedades que 
            squieren añadir y que se pueden representar en el mapa.

            geometry: objeto de tipo punto (latitud y longitud)
            id: identificador del objeto dentro de todo el gjson
            properties: conjunto de propiedades o datos que se quieren añadir para que puedan ser
            representados en el mapa'''
            feature_point = Feature(geometry = i_lnglat, id = j, 
                                    properties = {'mon': mon, 'inc': inc})
            # anañadiendo la figura geojson a la lista de objetos de tipo geojson
            lst_features.append(feature_point)
            # creando la coleccion para la lista de objetos tipo geojson
    return FeatureCollection(lst_features)
# invocando la funcion
cts_by_month = generate_centroids_by_month(df_centroids)

# creando el archivo
f = open("./data/crime/centroids_upz_nuse_month.geojson", "a")
f.write(str(cts_by_month))
f.close()